In [1]:
import pandas as pd
import ast
import os
from datetime import date, datetime, timedelta

import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
from shapely.geometry import Point
from sqlalchemy import create_engine

load_dotenv()

True

In [2]:
ani_code = ["0", "1"]
animal = ["dog", "cat"]

df = pd.DataFrame({
    "ani_code":ani_code,
    "animal":animal
})

df

,ani_code,animal
0,0,dog
1,1,cat


In [3]:
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df.to_sql(name="animal", con=engine, index=False, if_exists="replace")

2

In [ ]:
sql = "SELECT * FROM gmap_record"

df2 = pd.read_sql(sql, engine)



,city,coord_count,data_count,type,update_date
0,TPE,171,1610,salon,2025/10/29
1,TYU,98,1406,salon,2025/10/29
2,TCH,188,1471,salon,2025/10/29
3,TNA,186,1239,salon,2025/10/29
4,KSH,258,1353,salon,2025/10/29
5,TPE,225,2278,restaurant,2025/10/30
6,TYU,138,1557,restaurant,2025/10/30
7,TCH,250,1657,restaurant,2025/10/30
8,TNA,250,1903,restaurant,2025/10/30
9,KSH,349,1313,restaurant,2025/10/30


In [ ]:
df2["update_date"] = pd.to_datetime(df2["update_date"], format="mixed")

df2["update_date"] = df2["update_date"].dt.strftime("%Y/%m/%d")


,city,coord_count,data_count,type,update_date
0,TPE,171,1610,salon,2025/10/29
1,TYU,98,1406,salon,2025/10/29
2,TCH,188,1471,salon,2025/10/29
3,TNA,186,1239,salon,2025/10/29
4,KSH,258,1353,salon,2025/10/29
5,TPE,225,2278,restaurant,2025/10/30
6,TYU,138,1557,restaurant,2025/10/30
7,TCH,250,1657,restaurant,2025/10/30
8,TNA,250,1903,restaurant,2025/10/30
9,KSH,349,1313,restaurant,2025/10/30


In [8]:
df2.to_sql(name="gmap_record", con=engine, index=False, if_exists="replace")

30